### read daily xls into pandas

In [27]:
from pandas import DataFrame, ExcelFile
import os

In [28]:
xlss = [i for i in os.listdir() if i[:5] == 'KGAST']

In [29]:
xls = xlss[0]

In [31]:
df = ExcelFile(xls)

In [48]:
dff = df.parse()[['MITGLIEDER', 'KURZ', 'BESCHREIBUNG', 'VALOR', 'ANLAGEGRUPPE',
       'AUSGABEPREIS', 'RUECKNAHMEPREIS', 'AUFGEL_ERTRAG', 'EINGABESCHLUSS',
       'ABSCHLUSSTAG', 'VALUTA', 'AENDERUNGSDAT']]
dff.columns

Index(['MITGLIEDER', 'KURZ', 'BESCHREIBUNG', 'VALOR', 'ANLAGEGRUPPE',
       'AUSGABEPREIS', 'RUECKNAHMEPREIS', 'AUFGEL_ERTRAG', 'EINGABESCHLUSS',
       'ABSCHLUSSTAG', 'VALUTA', 'AENDERUNGSDAT'],
      dtype='object')

In [ ]:
a=dff.columns

In [47]:
dff[['MITGLIEDER', 'KURZ', 'BESCHREIBUNG', 'VALOR', 'ANLAGEGRUPPE',
       'AUSGABEPREIS', 'RUECKNAHMEPREIS', 'AUFGEL_ERTRAG', 'EINGABESCHLUSS',
       'ABSCHLUSSTAG', 'VALUTA', 'AENDERUNGSDAT']]

,MITGLIEDER,KURZ,BESCHREIBUNG,VALOR,ANLAGEGRUPPE,AUSGABEPREIS,RUECKNAHMEPREIS,AUFGEL_ERTRAG,EINGABESCHLUSS,ABSCHLUSSTAG,VALUTA,AENDERUNGSDAT
0,Anlagestiftung der UBS für Personalvorsorge / ...,UBS,NaN,10077053,UBS AST Kommerzielle Immobilien Schweiz,1512.67,1472.09,0,15:00 Uhr,2016-10-13,2016-10-17,2016-10-14
1,Zürich Anlagestiftung,Zürich,<p>&Auml;nderungsdatum = Handelstag<br />Absch...,1011761,Aktien Japan,744.37,742.14,0,12:00 h,2016-10-12,2016-10-17,2016-10-14
2,Credit Suisse Anlagestiftung / Credit Suisse A...,CSA / CSA 2,NaN,10157421,CSA Insurance Linked Stategies L,1431.01,1431.01,1.32,monthly,2016-08-31,2016-08-25,2016-10-14
3,J. Safra Sarasin Anlagestiftung,SAST,NaN,1016859,BVG-Rendite,1385,1385,12.58,10.00 Uhr,2016-10-12,2016-10-14,2016-10-13
4,J. Safra Sarasin Anlagestiftung,SAST,NaN,1016862,BVG-Nachhaltigkeit,1404,1404,-3.03,10.00 Uhr,2016-10-12,2016-10-14,2016-10-13
5,PRISMA Anlagestiftung,PRISMA,NaN,10197144,PRISMA Risk Budgeting Line 5,1'113.17,1'113.17,0,Do/jeudi 17:00,2016-10-07,2016-10-13,2016-10-11
6,Credit Suisse Anlagestiftung / Credit Suisse A...,CSA / CSA 2,NaN,10382676,CSA Mixta-BVG Index 45,1410.98,1407.18,-3.17,12:00 Uhr,2016-10-12,2016-10-17,2016-10-14
7,Credit Suisse Anlagestiftung / Credit Suisse A...,CSA / CSA 2,NaN,1039194,CSA Equity Small & Mid Cap Switzerland,2218.02,2218.02,0,14:30 Uhr,2016-10-13,2016-10-17,2016-10-14
8,Credit Suisse Anlagestiftung / Credit Suisse A...,CSA / CSA 2,NaN,10399033,CSA Global Bonds Hedged,1200.92,1200.92,-1.5,15:30 Uhr,2016-10-13,2016-10-17,2016-10-14
9,ASSETIMMO Immobilien-Anlagestiftung,assetimmo,"<p><span style=""color: #000000;""><strong>Hinwe...",1049345,Anlagegruppe W (Wohnliegenschaften),533.54,507.52,0,-,2016-09-30,2016-10-01,2016-10-03


In [37]:
dff.head(1)

,MITGLIEDER,KURZ,BESCHREIBUNG,VALOR,ANLAGEGRUPPE,AUSGABEPREIS,RUECKNAHMEPREIS,AUFGEL_ERTRAG,EINGABESCHLUSS,ABSCHLUSSTAG,VALUTA,AENDERUNGSDAT
0,Anlagestiftung der UBS für Personalvorsorge / ...,UBS,NaN,10077053,UBS AST Kommerzielle Immobilien Schweiz,1512.67,1472.09,0,15:00 Uhr,2016-10-13,2016-10-17,2016-10-14
